## Day 33 Lecture 2 Assignment

In this assignment, we will learn about non linear SVM models. We will use the heart disease dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ds_useful as ds

In [2]:
heart = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/framingham_heart_disease.csv')

In [3]:
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


This dataset helps us predict the probability of coronary heart diease (CHD) in the next 10 years given the risk factors for each subject in the study. Our target variable is `TenYearCHD`.

We'll start off by removing any rows containing missing data.

In [5]:
# answer below:

missings = ds.missingness_summary(heart, print_log=True)

glucose            9.155262
education          2.477584
BPMeds             1.250590
totChol            1.179802
cigsPerDay         0.684285
BMI                0.448325
heartRate          0.023596
TenYearCHD         0.000000
diaBP              0.000000
sysBP              0.000000
diabetes           0.000000
prevalentHyp       0.000000
prevalentStroke    0.000000
currentSmoker      0.000000
age                0.000000
male               0.000000
dtype: float64


In [30]:
heart.dropna(axis=0, inplace=True)
ds.missingness_summary(heart, print_log=True)
heart

TenYearCHD         0.0
glucose            0.0
heartRate          0.0
BMI                0.0
diaBP              0.0
sysBP              0.0
totChol            0.0
diabetes           0.0
prevalentHyp       0.0
prevalentStroke    0.0
BPMeds             0.0
cigsPerDay         0.0
currentSmoker      0.0
education          0.0
age                0.0
male               0.0
dtype: float64


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,1,58,3.0,0,0.0,0.0,0,1,0,187.0,141.0,81.0,24.96,80.0,81.0,0
4232,1,68,1.0,0,0.0,0.0,0,1,0,176.0,168.0,97.0,23.14,60.0,79.0,1
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0


We will then scale the data using the standard scaler. Do this in the cell below.

In [33]:
# answer below:
def get_categorical_columns(df, unique_thresh=10):
    columns = []
    for col in df.columns:
        if df[col].nunique() < unique_thresh:
            columns.append(col)
    return columns
cat_cols = get_categorical_columns(heart, unique_thresh = 2.1)

print(heart.shape)
print(heart[cat_cols].shape)
num_heart = heart.drop(cat_cols, axis=1)
print(num_heart.shape)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
num_heart_s = pd.DataFrame(scaler.fit_transform(num_heart), columns=num_heart.columns)
num_heart_s

(3656, 16)
(3656, 7)
(3656, 9)


,age,education,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,-1.233351,1.975752,-0.757068,-0.949714,-1.193695,-1.078415,0.291688,0.356340,-0.203127
1,-0.415591,0.019795,-0.757068,0.297729,-0.514637,-0.159695,0.724614,1.608289,-0.244956
2,-0.181945,-0.958183,0.921174,0.184325,-0.220378,-0.243215,-0.109261,-0.060977,-0.495930
3,1.336754,0.997773,1.760294,-0.269291,0.798209,1.009584,0.687717,-0.895610,0.884427
4,-0.415591,0.997773,1.172910,1.091556,-0.107202,0.090864,-0.660258,0.773656,0.131505
...,...,...,...,...,...,...,...,...,...
3651,0.986285,0.997773,-0.757068,-1.131161,0.390774,-0.159695,-0.202734,0.356340,-0.035811
3652,2.154514,-0.958183,-0.757068,-1.380649,1.613079,1.176624,-0.650419,-1.312926,-0.119469
3653,0.051701,-0.958183,-0.673156,1.726618,2.111055,0.759024,0.045707,-0.812146,0.173334
3654,0.168524,0.997773,2.851151,-0.677545,-0.265649,-0.243215,-1.494133,-0.895610,-0.579588


In [35]:
heart_s = pd.concat([heart[cat_cols].reset_index(drop=True), num_heart_s], axis=1)
heart_s

,male,currentSmoker,BPMeds,prevalentStroke,prevalentHyp,diabetes,TenYearCHD,age,education,cigsPerDay,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,0,0.0,0,0,0,0,-1.233351,1.975752,-0.757068,-0.949714,-1.193695,-1.078415,0.291688,0.356340,-0.203127
1,0,0,0.0,0,0,0,0,-0.415591,0.019795,-0.757068,0.297729,-0.514637,-0.159695,0.724614,1.608289,-0.244956
2,1,1,0.0,0,0,0,0,-0.181945,-0.958183,0.921174,0.184325,-0.220378,-0.243215,-0.109261,-0.060977,-0.495930
3,0,1,0.0,0,1,0,1,1.336754,0.997773,1.760294,-0.269291,0.798209,1.009584,0.687717,-0.895610,0.884427
4,0,1,0.0,0,0,0,0,-0.415591,0.997773,1.172910,1.091556,-0.107202,0.090864,-0.660258,0.773656,0.131505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,1,0,0.0,0,1,0,0,0.986285,0.997773,-0.757068,-1.131161,0.390774,-0.159695,-0.202734,0.356340,-0.035811
3652,1,0,0.0,0,1,0,1,2.154514,-0.958183,-0.757068,-1.380649,1.613079,1.176624,-0.650419,-1.312926,-0.119469
3653,1,1,0.0,0,1,0,1,0.051701,-0.958183,-0.673156,1.726618,2.111055,0.759024,0.045707,-0.812146,0.173334
3654,1,1,0.0,0,0,0,0,0.168524,0.997773,2.851151,-0.677545,-0.265649,-0.243215,-1.494133,-0.895610,-0.579588


Finally, we wil split the data into train and test with 20% of the data in the test subset.

In [37]:
# answer below:
from sklearn.model_selection import train_test_split

X = heart_s.drop('TenYearCHD', axis=1)
Y = heart_s['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=123)

In [38]:
# We would like to compare the runtime of a number of functions later in our code. 
import time

Generate a polynomial SVC model and a RBF SVC model. Compare the score for the two models on the test data and also compare the runtime of the two models.

In [47]:
# answer below:
from sklearn.svm import SVC

c = 1

start_time = time.time()
svm_poly = SVC(kernel='poly', C=c)
svm_poly.fit(X_train, Y_train)
end_time = time.time()

print('SVC with polynomial kernel (c={}):'.format(c))
print('Time:           {}'.format(end_time - start_time))
print('Training Score: {}'.format(svm_poly.score(X_train, Y_train)))
print('Testing Score:  {}'.format(svm_poly.score(X_test, Y_test)))

SVC with polynomial kernel (c=1):
Time:           0.5059285163879395
Training Score: 0.8679890560875513
Testing Score:  0.8401639344262295


In [45]:
c = 1

start_time = time.time()
svm_rbf = SVC(kernel='rbf', C=c)
svm_rbf.fit(X_train, Y_train)
end_time = time.time()

print('SVC with radial basis function kernel (c={}):'.format(c))
print('Time:           {}'.format(end_time - start_time))
print('Training Score: {}'.format(svm_rbf.score(X_train, Y_train)))
print('Testing Score:  {}'.format(svm_rbf.score(X_test, Y_test)))

SVC with radial basis function kernel (c=1):
Time:           0.2264847755432129
Training Score: 0.859781121751026
Testing Score:  0.8456284153005464


Which model overfits more? Why? How would you improve the model that overfits more?

In [0]:
# answer below:

The model that overfits more is the polynomial model. It does this because it is using a 3D polynomial to segregate data that may not need to be segregated that highly. Of course, you could either lower the dimension, which may cost accuracy, or you could simply regularize the cost function a bit more with a lower c value. We'll try both!

In [51]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(svm_poly, param_grid={'C':[.0001, .001, .1], 'degree': [2,3]})
gs.fit(X_train, Y_train)

print(gs.best_params_)
print(gs.best_score_)

{'C': 0.1, 'degree': 3}
0.8498635991101745


In [52]:
c = .1

start_time = time.time()
svm_poly = SVC(kernel='poly', C=c, degree=3)
svm_poly.fit(X_train, Y_train)
end_time = time.time()

print('SVC with polynomial kernel (c={}):'.format(c))
print('Time:           {}'.format(end_time - start_time))
print('Training Score: {}'.format(svm_poly.score(X_train, Y_train)))
print('Testing Score:  {}'.format(svm_poly.score(X_test, Y_test)))

SVC with polynomial kernel (c=0.1):
Time:           0.2414085865020752
Training Score: 0.8570451436388509
Testing Score:  0.8415300546448088


Turns out all we needed was some regularization